## Tail Recursion

Recursive function calls can be _tail recursive_ if the last call of the function can be returns immediately. Scala will convert this into a while loop, and there will be not stack frame exhaustion. An example of a recursive call that is not tail recursive:
```scala
1 + go(n, acc)
```
In this case, even after having evaluated `go(n, acc)` the funcion result must be returned in order to be evaluated in the stack above. 
A _tail call elimination_ can be made when there is no subsequent evaluation.

An annotation `@annotation.tailrec` can be added before the function call in order to catch any recursive functions that should but cannot be _tail call eliminated_.

### Exercise 2.1

In [2]:
 def fibTail(n: Int): Int = {

    @annotation.tailrec
    def go(n: Int, current: Int, fibCurrent: Int, fibPrevious: Int): Int = {

      if(n == current) return(fibCurrent)

      go(n, current + 1, fibCurrent + fibPrevious, fibCurrent)
    }

    go(n, 1, 1, 0)

  }


defined function fibTail

In [3]:
fibTail(10)

res2: Int = 55

## Polymorphic functions
* _monomorphic functions_ operate on one datatype
* _polymorphic functions_ operate on any datatype. This is a special case of polymorphism often referred to as _parametric polymorphism_.


### Exercise 2.2

In [2]:
def isSorted[A](as: Array[A], ordered: (A,A) => Boolean): Boolean = {
    @annotation.tailrec
    def loop(n: Int): Boolean = {
        if(n >= as.length) true
        else if(ordered(as(n-1), as(n))) loop(n+1)
        else false
    }

    if(as.length == 1) return(true)
    
    loop(1)
}

defined function isSorted

In [6]:
isSorted(Array(1), (x: Int,y: Int) => x <= y)

res3: Boolean = true

In [7]:
isSorted(Array(1,2,3), (x: Int,y: Int) => x <= y)

res4: Boolean = true

In [11]:
isSorted(Array(1,6,5), (x: Int,y: Int) => x <= y)

res8: Boolean = false

In [16]:
isSorted(Array("AB", "B"), (x: String, y: String) => x <= y)

res12: Boolean = true

### Functions as values
When we define a function literal like `(a, b) => a < b` it's really just synctatic sugar for:
```scala
val lessThan = new Function2[Int, Int, Boolean] { 
    def apply(a: Int, b: Int) = a < b
}
```
`Function2` is a scala _trait_ (similar to interface or rather mixin), and has a function `apply` defined.

## Partially applied functions
### Exercise 2.3

In [18]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) = {
    (a: A) => (b: B) => f(a,b)
}

defined function curry

In [22]:
def add(x: Int, y: Int) = x + y

defined function add

In [23]:
add(2,3)

res18: Int = 5

In [27]:
val add3to = curry(add)(3)

add3to: Int => Int = <function1>

In [28]:
add3to(4)

res23: Int = 7

### Exercise 2.4

In [29]:
def uncurry[A,B,C](f: A => B => C): (A, B) => C = {
    (a: A, b: B) => f(a)(b)
}

defined function uncurry

In [34]:
val uncurriedAdd = uncurry(curry(add))

uncurriedAdd: (Int, Int) => Int = <function2>

In [36]:
uncurriedAdd(2,3)

res29: Int = 5

### Exercise 2.5

In [37]:
def compose[A, B, C](f: B => C, g: A => B): A => C = {
    (a: A) => f(g(a))
}

defined function compose

In [38]:
def addQuotes(d: Int): String = "'" + d + "'"

defined function addQuotes

In [39]:
def add3AndCompose = compose(addQuotes, add3to)

defined function add3AndCompose

In [40]:
add3AndCompose(4)

res33: String = "'7'"